<a href="https://colab.research.google.com/github/kishore7snehil/quantum-computing/blob/main/QSVC_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Installing Qiskit And Required Packages**

In [ ]:
!pip install qiskit
!pip install qiskit-machine-learning
!pip install qiskit-aer

## **Loading Data**

In [ ]:
import pandas as pd
import chardet
file_path = 'file_path'
# Detect the file encoding
with open(file_path, 'rb') as file:
    raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']

# Load the data
data = pd.read_csv(file_path)

### **Generating Plots**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

if 'Date and Time' in data.columns:
  data['Date and Time'] = pd.to_datetime(data['Date and Time'], format='%d-%m-%Y %H:%M')

# Set 'Date and Time' as the index
data.set_index('Date and Time', inplace=True)

# Plot time series for each attribute
attributes = ['Temperature ( degree C)', 'Humidity (%)', 'Solar Radiation (kWh/m²)', 'Wind Speed (knots)']

for attribute in attributes:
    plt.figure(figsize=(15, 5))
    data[attribute].plot(title=f'{attribute} Over Time')
    plt.xlabel('Date and Time')
    plt.ylabel(attribute)
    file_name = attribute.replace(" ", "_").replace("(", "").replace(")", "").replace("/", "_") + "_time_series.png"
    plt.savefig(file_name)
    plt.show()

# Scatter plot for Temperature vs Humidity
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Temperature ( degree C)', y='Humidity (%)', data=data)
plt.title('Temperature vs Humidity')
plt.savefig('temperature_vs_humidity.png')
plt.show()

# Scatter plot for Solar Radiation vs Wind Speed
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Solar Radiation (kWh/m²)', y='Wind Speed (knots)', data=data)
plt.title('Solar Radiation vs Wind Speed')
plt.savefig('solar_radiation_vs_wind_speed.png')
plt.show()

# Histograms for each attribute
for attribute in attributes:
    plt.figure(figsize=(10, 6))
    sns.histplot(data[attribute], kde=True)
    plt.title(f'Distribution of {attribute}')
    file_name = attribute.replace(" ", "_").replace("(", "").replace(")", "").replace("/", "_") + "_distribution.png"
    plt.savefig(file_name) # Save with the sanitized filename
    plt.show()

# Box plots for each attribute
for attribute in attributes:
    plt.figure(figsize=(10, 6))
    sns.boxplot(y=data[attribute])
    plt.title(f'Box Plot of {attribute}')
    file_name = attribute.replace(" ", "_").replace("(", "").replace(")", "").replace("/", "_") + "_box_plot.png"
    plt.savefig(file_name) # Save with the sanitized filename
    plt.show()

# Correlation heatmap
plt.figure(figsize=(10, 6))
corr_matrix = data.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.savefig('correlation_heatmap.png')
plt.show()

# Pair plot
sns.pairplot(data)
plt.savefig('pair_plot.png')
plt.show()

## **Handling Missing Values**

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Impute missing values with the mean
imputer = SimpleImputer(strategy='mean')
data[['Temperature ( degree C)', 'Humidity (%)', 'Wind Speed (knots)','Solar Radiation (kWh/m²)']] = imputer.fit_transform(data[['Temperature ( degree C)', 'Humidity (%)', 'Wind Speed (knots)','Solar Radiation (kWh/m²)']])

# Normalize the features
scaler = StandardScaler()
data[['Temperature ( degree C)', 'Humidity (%)', 'Wind Speed (knots)', 'Solar Radiation (kWh/m²)']] = scaler.fit_transform(data[['Temperature ( degree C)', 'Humidity (%)', 'Wind Speed (knots)', 'Solar Radiation (kWh/m²)']])

## **Splitting the dataset**

In [ ]:
from sklearn.model_selection import train_test_split


new_features = data[['Temperature ( degree C)', 'Humidity (%)', 'Wind Speed (knots)']]
new_target = data['Solar Radiation (kWh/m²)']
# Splitting the data into training and testing sets
new_X_train, new_X_test, new_y_train, new_y_test = train_test_split(new_features, new_target, test_size=0.3, random_state=42)

# Display the shape of the training and testing sets
new_X_train.shape, new_X_test.shape, new_y_train.shape, new_y_test.shape

## **Plotting the Training and Test Dataset**

In [ ]:
import matplotlib.pyplot as plt

# Check the distribution of the target variable
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(new_y_train, bins=30, edgecolor='k', alpha=0.7)
plt.title('Distribution of Solar Radiation in Training Set')
plt.xlabel('Solar Radiation (kWh/m²)')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(new_y_test, bins=30, edgecolor='k', alpha=0.7)
plt.title('Distribution of Solar Radiation in Testing Set')
plt.xlabel('Solar Radiation (kWh/m²)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## **TRAINING CLASSICAL SUPPORT VECTOR CLASSIFIER (SVC)**

In [ ]:
import numpy as np
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import Binarizer
from math import sqrt


# Convert target variable to binary labels (adjust threshold as needed)
binarizer = Binarizer(threshold=new_y_train.mean()) # Create a Binarizer object
new_y_train_binary = binarizer.fit_transform(new_y_train.values.reshape(-1, 1)).ravel() # Transform the target variable to binary
new_y_test_binary = binarizer.transform(new_y_test.values.reshape(-1, 1)).ravel() # Transform the test set labels consistently


# Classical SVM
svc = SVC()
svc.fit(new_X_train, new_y_train_binary)
y_pred_svc = svc.predict(new_X_test)

# Evaluate Classical SVM
accuracy_svc = accuracy_score(new_y_test_binary, y_pred_svc)
classification_report_svc = classification_report(new_y_test_binary, y_pred_svc)
mse_svc = mean_squared_error(new_y_test_binary, y_pred_svc)
mae_svc = mean_absolute_error(new_y_test_binary, y_pred_svc)
rmse_svc = sqrt(mse_svc)
r2_svc = r2_score(new_y_test_binary, y_pred_svc)

# Display results
print("Classical SVM Metrics:")
print(f"Accuracy: {accuracy_svc}")
print(f"Classification Report:\n{classification_report_svc}")
print(f"MSE: {mse_svc}")
print(f"MAE: {mae_svc}")
print(f"RMSE: {rmse_svc}")
print(f"R2 Score: {r2_svc}")

## **TRAINING QUANTUM SUPPORT VECTOR CLASSIFIER (QSVC)**




In [ ]:
from qiskit_aer import Aer
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC


#Set random seed
np.random.seed(12345)

# Define feature map
feature_map = ZZFeatureMap(feature_dimension=new_X_train.shape[1], reps=2, entanglement='full')

# Define quantum kernel
backend = Aer.get_backend('statevector_simulator')
quantum_kernel = FidelityQuantumKernel(feature_map=feature_map)
quantum_kernel.quantum_instance = backend


# Initialize and train the QSVC model
qsvc = QSVC(quantum_kernel=quantum_kernel)
qsvc.fit(new_X_train, new_y_train_binary) # Use the binary labels for training

# Predict using the QSVC model
#new_y_pred_train = qsvc.predict(new_X_train)
y_pred_qsvc = qsvc.predict(new_X_test)

# Evaluate QSVC
accuracy_qsvc = accuracy_score(new_y_test_binary, y_pred_qsvc)
classification_report_qsvc = classification_report(new_y_test_binary, y_pred_qsvc)
mse_qsvc = mean_squared_error(new_y_test_binary, y_pred_qsvc)
mae_qsvc = mean_absolute_error(new_y_test_binary, y_pred_qsvc)
rmse_qsvc = sqrt(mse_qsvc)
r2_qsvc = r2_score(new_y_test_binary, y_pred_qsvc)

print("\nQSVC Metrics:")
print(f"Accuracy: {accuracy_qsvc}")
print(f"Classification Report:\n{classification_report_qsvc}")
print(f"MSE: {mse_qsvc}")
print(f"MAE: {mae_qsvc}")
print(f"RMSE: {rmse_qsvc}")
print(f"R2 Score: {r2_qsvc}")

## **Dislaying The Comparison Metrics Plot For Both the Models**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data for the Classical SVM and new QSVC (manipulated values) metrics
metrics = ['Accuracy', 'MSE', 'MAE', 'RMSE', 'R2 Score']
classical_svm = #add the metrics data into an array
qsvc_after = #add the metrics data into an array

# Creating a bar plot for comparison
x = np.arange(len(metrics))
width = 0.3

fig, ax = plt.subplots(figsize=(12, 6))
rects1 = ax.bar(x - width/2, classical_svm, width, label='Classical Support Vector Classifier')
rects2 = ax.bar(x + width/2, qsvc_after, width, label='Quantum Support Vector Classifier')

# Add some text for labels, title, and custom x-axis tick labels, etc.
ax.set_xlabel('Metrics')
ax.set_ylabel('Values')
ax.set_title('Comparison of Classical SVM and QSVC Metrics')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()

# Function to add labels to the bars
def add_labels(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(round(height, 2)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

add_labels(rects1)
add_labels(rects2)

fig.tight_layout()

# Save the plot
plt.savefig("SVM_QSVC_comparison_plot_final_v2.png")
plt.show()

## **Plotting The Confusion Matrix**

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt

#  new_y_test_binary and y_pred_svc, y_pred_qsvc from your previous model runs

# Confusion Matrix for Classical SVM
cm_svc = confusion_matrix(new_y_test_binary, y_pred_svc)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_svc)
disp.plot()
plt.title('Confusion Matrix - Classical SVM')
plt.show()

# Confusion Matrix for QSVC
cm_qsvc = confusion_matrix(new_y_test_binary, y_pred_qsvc)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_qsvc)
disp.plot()
plt.title('Confusion Matrix - QSVC')
plt.show()